##CV features

In [1]:
import re
import logging 
import gensim
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import xgboost as xgb 
from time import time 
import pandas as pd
from tqdm import tqdm 
from functools import reduce
from sklearn.manifold import TSNE
from collections import defaultdict
from gensim.models import FastText
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

ImportError: No module named 'sklearn.__check_build._check_build'
___________________________________________________________________________
Contents of c:\Users\jyoti\anaconda3\envs\newInsta\lib\site-packages\sklearn\__check_build:
setup.py                  _check_build.cp310-win_amd64.pyd__init__.py
__pycache__
___________________________________________________________________________
It seems that scikit-learn has not been built correctly.

If you have installed scikit-learn from source, please do not forget
to build the package before using it: run `python setup.py install` or
`make` in the source directory.

If you have used an installer, please check that it is suited for your
Python version, your operating system and your platform.

In [3]:
path='C:\\Users\\jyoti\\Desktop\\NLP\\NLP_project\\NLP-project-\\'

In [65]:
df = pd.read_csv(path+ "Dataset\\df_final_cv.csv")

C:\Users\jyoti\anaconda3\envs\nlp\lib\site-packages\IPython\core\interactiveshell.py:3229: DtypeWarning: Columns (49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# df.drop(['Unnamed: 0'], axis = 1,inplace=True)

In [63]:
df.drop(df.columns.difference(['tags','confidence_score','accent_color','is_bw','dominant_colors','bg_color','fore_color','likes']), 1, inplace=True)

In [64]:
df

,likes,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,29.0,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","(0.788235294117647, 0.00784313725490196, 0.007...",0,"[(1.0, 0.7529411764705882, 0.796078431372549)]","(1.0, 0.7529411764705882, 0.796078431372549)","(1.0, 0.7529411764705882, 0.796078431372549)"
1,44.0,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","(0.6901960784313725, 0.14901960784313725, 0.10...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.0, 0.0, 0.0)"
2,209.0,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...","(0.6588235294117647, 0.4392156862745098, 0.141...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.6470588235294118, 0.16470588235294117, 0.16..."
3,95.0,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...","(0.24313725490196078, 0.12549019607843137, 0.0...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.6470588235294118, 0.16470588235294117, 0.16..."
4,101.0,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...","(0.792156862745098, 0.7372549019607844, 0.0039...",0,"[(1.0, 1.0, 0.0)]","(1.0, 1.0, 0.0)","(1.0, 1.0, 0.0)"
...,...,...,...,...,...,...,...,...
88511,8.0,"['text', 'food', 'snack']","[0.9999812841415405, 0.9580191373825073, 0.915...","(0.6588235294117647, 0.00784313725490196, 0.71...",0,"[(0.5019607843137255, 0.0, 0.5019607843137255)...","(0.5019607843137255, 0.0, 0.5019607843137255)","(0.5019607843137255, 0.5019607843137255, 0.501..."
88512,9.0,"['text', 'poster', 'cartoon']","[0.9999863505363464, 0.9184243679046631, 0.910...","(0.01568627450980392, 0.5098039215686274, 0.78...",0,"[(1.0, 1.0, 1.0)]","(1.0, 1.0, 1.0)","(1.0, 1.0, 1.0)"
88513,11.0,"['text', 'yellow', 'packaging and labeling', '...","[0.9997624158859253, 0.9444339871406555, 0.854...","(0.01568627450980392, 0.2235294117647059, 0.43...",0,"[(1.0, 1.0, 0.0), (0.0, 0.0, 1.0)]","(0.0, 0.0, 1.0)","(1.0, 1.0, 0.0)"
88514,NaN,"['sport', 'person', 'barbell', 'physical fitne...","[0.9959811568260193, 0.9808116555213928, 0.971...","(0.6862745098039216, 0.5215686274509804, 0.109...",0,"[(1.0, 1.0, 1.0)]","(1.0, 1.0, 1.0)","(1.0, 1.0, 1.0)"


In [9]:
#calculating missing values in the dataset
missing_values = df.isnull().sum()
missing_per = (missing_values/df.shape[0])*100
missing_table = pd.concat([missing_values,missing_per], axis=1, ignore_index=True) 
missing_table.rename(columns={0:'Total Missing Values',1:'Missing %'}, inplace=True)
missing_table

,Total Missing Values,Missing %
likes,2,0.002259
tags,0,0.000000
confidence_score,0,0.000000
accent_color,0,0.000000
is_bw,0,0.000000
dominant_colors,0,0.000000
bg_color,0,0.000000
fore_color,0,0.000000


In [10]:
#fill the missing values with zero
df['likes'] = df['likes'].fillna(0)

In [11]:
df1=df[:]

In [12]:
REPLACE_BY_SPACE_RE = re.compile('[^a-zA-Z_0-9,.]')

def strip_text(text: str) -> str:
    """
    Removes any left or right spacing (including carriage return) from text.
    Example:
    Input: '  This assignment is cool\n'
    Output: 'This assignment is cool'
    """

    return text.strip()


def replace_br(text: str) -> str:
    """
    Replaces br characters
    """

    return text.replace("'", "")


def replace_special_characters(text: str) -> str:
    """
    Replaces special characters, such as paranthesis,
    with spacing character
    """

    return REPLACE_BY_SPACE_RE.sub(' ', text)


from typing import List, Callable, Dict

PREPROCESSING_PIPELINE = [
                          replace_special_characters,
                          strip_text
                          ]



def text_prepare(text: str, filter_methods: List[Callable[[str], str]] = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """
    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE
    return reduce(lambda txt, f: f(txt), filter_methods, text)

# Pre-processing

print('Pre-processing text...')
print('[Debug] Before:\n{}'.format(df1.tags[:10]))

L=['tags','confidence_score','accent_color','dominant_colors','bg_color','fore_color']

A=['tags']

for i in L:
  df1[i]=df1[i].apply(lambda txt: text_prepare(str(txt)[1:-1]))



print('[Debug] After:\n{}'.format(df1.tags[:10]))
print()

print("Pre-processing completed!")

Pre-processing text...
[Debug] Before:
0    ['text', 'font', 'graphics', 'screenshot', 'gr...
1    ['dessert', 'baked goods', 'baking', 'snack', ...
2    ['food', 'fruit', 'baked goods', 'breakfast', ...
3    ['food', 'bread', 'fast food', 'bun', 'america...
4    ['text', 'font', 'design', 'yellow', 'graphics...
5    ['snack', 'baked goods', 'baking', 'dessert', ...
6    ['dessert', 'baked goods', 'baking', 'sweetnes...
7    ['text', 'food', 'shelf', 'countertop', 'indoo...
8    ['text', 'table', 'food', 'bakery', 'clothing'...
9    ['baked goods', 'sweetness', 'food', 'baking',...
Name: tags, dtype: object
[Debug] After:
0    text ,  font ,  graphics ,  screenshot ,  grap...
1    dessert ,  baked goods ,  baking ,  snack ,  f...
2    food ,  fruit ,  baked goods ,  breakfast ,  c...
3    food ,  bread ,  fast food ,  bun ,  american ...
4    text ,  font ,  design ,  yellow ,  graphics ,...
5    snack ,  baked goods ,  baking ,  dessert ,  b...
6    dessert ,  baked goods ,  baking , 

In [13]:
df1

,likes,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,29.0,"text , font , graphics , screenshot , grap...","0.9980798959732056, 0.9481294751167297, 0.8818...","0.788235294117647, 0.00784313725490196, 0.0078...",0,"1.0, 0.7529411764705882, 0.796078431372549","1.0, 0.7529411764705882, 0.796078431372549","1.0, 0.7529411764705882, 0.796078431372549"
1,44.0,"dessert , baked goods , baking , snack , f...","0.9897554516792297, 0.987897515296936, 0.98287...","0.6901960784313725, 0.14901960784313725, 0.105...",0,"0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164...","0.0, 0.0, 0.0"
2,209.0,"food , fruit , baked goods , breakfast , c...","0.9808361530303955, 0.9546540379524231, 0.9477...","0.6588235294117647, 0.4392156862745098, 0.1411...",0,"0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164..."
3,95.0,"food , bread , fast food , bun , american ...","0.9958561658859253, 0.981575608253479, 0.97992...","0.24313725490196078, 0.12549019607843137, 0.07...",0,"0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164..."
4,101.0,"text , font , design , yellow , graphics ,...","0.9993002414703369, 0.9640201926231384, 0.9513...","0.792156862745098, 0.7372549019607844, 0.00392...",0,"1.0, 1.0, 0.0","1.0, 1.0, 0.0","1.0, 1.0, 0.0"
...,...,...,...,...,...,...,...,...
88511,8.0,"text , food , snack","0.9999812841415405, 0.9580191373825073, 0.9157...","0.6588235294117647, 0.00784313725490196, 0.713...",0,"0.5019607843137255, 0.0, 0.5019607843137255 , ...","0.5019607843137255, 0.0, 0.5019607843137255","0.5019607843137255, 0.5019607843137255, 0.5019..."
88512,9.0,"text , poster , cartoon","0.9999863505363464, 0.9184243679046631, 0.9104...","0.01568627450980392, 0.5098039215686274, 0.780...",0,"1.0, 1.0, 1.0","1.0, 1.0, 1.0","1.0, 1.0, 1.0"
88513,11.0,"text , yellow , packaging and labeling , fo...","0.9997624158859253, 0.9444339871406555, 0.8540...","0.01568627450980392, 0.2235294117647059, 0.439...",0,"1.0, 1.0, 0.0 , 0.0, 0.0, 1.0","0.0, 0.0, 1.0","1.0, 1.0, 0.0"
88514,0.0,"sport , person , barbell , physical fitness...","0.9959811568260193, 0.9808116555213928, 0.9714...","0.6862745098039216, 0.5215686274509804, 0.1098...",0,"1.0, 1.0, 1.0","1.0, 1.0, 1.0","1.0, 1.0, 1.0"


In [14]:
df1

,likes,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,29.0,"text , font , graphics , screenshot , grap...","0.9980798959732056, 0.9481294751167297, 0.8818...","0.788235294117647, 0.00784313725490196, 0.0078...",0,"1.0, 0.7529411764705882, 0.796078431372549","1.0, 0.7529411764705882, 0.796078431372549","1.0, 0.7529411764705882, 0.796078431372549"
1,44.0,"dessert , baked goods , baking , snack , f...","0.9897554516792297, 0.987897515296936, 0.98287...","0.6901960784313725, 0.14901960784313725, 0.105...",0,"0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164...","0.0, 0.0, 0.0"
2,209.0,"food , fruit , baked goods , breakfast , c...","0.9808361530303955, 0.9546540379524231, 0.9477...","0.6588235294117647, 0.4392156862745098, 0.1411...",0,"0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164..."
3,95.0,"food , bread , fast food , bun , american ...","0.9958561658859253, 0.981575608253479, 0.97992...","0.24313725490196078, 0.12549019607843137, 0.07...",0,"0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164..."
4,101.0,"text , font , design , yellow , graphics ,...","0.9993002414703369, 0.9640201926231384, 0.9513...","0.792156862745098, 0.7372549019607844, 0.00392...",0,"1.0, 1.0, 0.0","1.0, 1.0, 0.0","1.0, 1.0, 0.0"
...,...,...,...,...,...,...,...,...
88511,8.0,"text , food , snack","0.9999812841415405, 0.9580191373825073, 0.9157...","0.6588235294117647, 0.00784313725490196, 0.713...",0,"0.5019607843137255, 0.0, 0.5019607843137255 , ...","0.5019607843137255, 0.0, 0.5019607843137255","0.5019607843137255, 0.5019607843137255, 0.5019..."
88512,9.0,"text , poster , cartoon","0.9999863505363464, 0.9184243679046631, 0.9104...","0.01568627450980392, 0.5098039215686274, 0.780...",0,"1.0, 1.0, 1.0","1.0, 1.0, 1.0","1.0, 1.0, 1.0"
88513,11.0,"text , yellow , packaging and labeling , fo...","0.9997624158859253, 0.9444339871406555, 0.8540...","0.01568627450980392, 0.2235294117647059, 0.439...",0,"1.0, 1.0, 0.0 , 0.0, 0.0, 1.0","0.0, 0.0, 1.0","1.0, 1.0, 0.0"
88514,0.0,"sport , person , barbell , physical fitness...","0.9959811568260193, 0.9808116555213928, 0.9714...","0.6862745098039216, 0.5215686274509804, 0.1098...",0,"1.0, 1.0, 1.0","1.0, 1.0, 1.0","1.0, 1.0, 1.0"


In [16]:
# for i in df1['confidence_score']:

def convert_str2int(i):
  n = []
  y = i.split(', ')
  if len(y[0]) == 0:
    return []
  for j in range(len(y)):
    n.append(float(y[j]))
  return n

df1['confidence_score'] = df1['confidence_score'].apply(convert_str2int)
df1['accent_color'] = df1['accent_color'].apply(convert_str2int)
df1['dominant_colors'] = df1['dominant_colors'].apply(convert_str2int)
df1['bg_color'] = df1['bg_color'].apply(convert_str2int)
df1['fore_color'] = df1['fore_color'].apply(convert_str2int)

In [17]:
df1

,likes,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,29.0,"text , font , graphics , screenshot , grap...","[0.9980798959732056, 0.9481294751167297, 0.881...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]"
1,44.0,"dessert , baked goods , baking , snack , f...","[0.9897554516792297, 0.987897515296936, 0.9828...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]"
2,209.0,"food , fruit , baked goods , breakfast , c...","[0.9808361530303955, 0.9546540379524231, 0.947...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16..."
3,95.0,"food , bread , fast food , bun , american ...","[0.9958561658859253, 0.981575608253479, 0.9799...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16..."
4,101.0,"text , font , design , yellow , graphics ,...","[0.9993002414703369, 0.9640201926231384, 0.951...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]"
...,...,...,...,...,...,...,...,...
88511,8.0,"text , food , snack","[0.9999812841415405, 0.9580191373825073, 0.915...","[0.6588235294117647, 0.00784313725490196, 0.71...",0,"[0.5019607843137255, 0.0, 0.5019607843137255, ...","[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.5019607843137255, 0.501..."
88512,9.0,"text , poster , cartoon","[0.9999863505363464, 0.9184243679046631, 0.910...","[0.01568627450980392, 0.5098039215686274, 0.78...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]"
88513,11.0,"text , yellow , packaging and labeling , fo...","[0.9997624158859253, 0.9444339871406555, 0.854...","[0.01568627450980392, 0.2235294117647059, 0.43...",0,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[1.0, 1.0, 0.0]"
88514,0.0,"sport , person , barbell , physical fitness...","[0.9959811568260193, 0.9808116555213928, 0.971...","[0.6862745098039216, 0.5215686274509804, 0.109...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]"


In [18]:
def convert_str2list(i):
  n = []
  y = i.split(', ')
  for j in range(len(y)):
    n.append(str(y[j]))
  return n
df1['tags'] = df['tags'].apply(convert_str2list)
df1['tags'][0]

['text ',
 ' font ',
 ' graphics ',
 ' screenshot ',
 ' graphic design ',
 ' design ',
 ' typography']

In [19]:
df1['confidence_score'][6]

[0.9938297271728516,
 0.9784528613090515,
 0.977806806564331,
 0.966992974281311,
 0.9616279602050781,
 0.9515609741210938,
 0.9280351400375366,
 0.912075936794281,
 0.9083282947540283,
 0.9079588651657104,
 0.8951475620269775,
 0.8874362707138062,
 0.8836393356323242,
 0.8746436834335327,
 0.8713182210922241,
 0.8668385744094849,
 0.8426526784896851,
 0.786042332649231,
 0.5681669116020203]

In [20]:
def trim_lowConfidence_tags(tag_column):
  for scoreList in df1['confidence_score']:
    for score_index in range(len(scoreList)):
      if scoreList[score_index]<0.5:
        return tag_column[:score_index]

df1['tags'] = df1['tags'].apply(trim_lowConfidence_tags)  

In [21]:
df1.drop('confidence_score',1,inplace=True)

In [22]:
df1.dominant_colors[1]

[0.6470588235294118,
 0.16470588235294117,
 0.16470588235294117,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0]

In [23]:
df1['dominant_colors'][:3]

0         [1.0, 0.7529411764705882, 0.796078431372549]
1    [0.6470588235294118, 0.16470588235294117, 0.16...
2    [0.6470588235294118, 0.16470588235294117, 0.16...
Name: dominant_colors, dtype: object

In [24]:
def separate_column(column):
  if len(column)>0:
    return column[0]
  else:
    return []

df1['dominant_colors_R'] = df1['dominant_colors'].apply(separate_column)  
df1['accent_color_R'] = df1['accent_color'].apply(separate_column)  
df1['bg_color_R'] = df1['bg_color'].apply(separate_column)  
df1['fore_color_R'] = df1['fore_color'].apply(separate_column)  

In [25]:
def separate_column(column):
  if len(column)>0:
    return column[1]
  else:
    return []

df1['dominant_colors_G'] = df1['dominant_colors'].apply(separate_column)  
df1['accent_color_G'] = df1['accent_color'].apply(separate_column)  
df1['bg_color_G'] = df1['bg_color'].apply(separate_column)  
df1['fore_color_G'] = df1['fore_color'].apply(separate_column)  

In [26]:
def separate_column(column):
  if len(column)>0:
    return column[2]
  else:
    return []

df1['dominant_colors_B'] = df1['dominant_colors'].apply(separate_column)  
df1['accent_color_B'] = df1['accent_color'].apply(separate_column)  
df1['bg_color_B'] = df1['bg_color'].apply(separate_column)  
df1['fore_color_B'] = df1['fore_color'].apply(separate_column)  

In [27]:
df1

,likes,tags,accent_color,is_bw,dominant_colors,bg_color,fore_color,dominant_colors_R,accent_color_R,bg_color_R,fore_color_R,dominant_colors_G,accent_color_G,bg_color_G,fore_color_G,dominant_colors_B,accent_color_B,bg_color_B,fore_color_B
0,29.0,"[text , font , graphics , screenshot , gra...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]",1,0.788235,1.000000,1.000000,0.752941,0.007843,0.752941,0.752941,0.796078,0.007843,0.796078,0.796078
1,44.0,"[dessert , baked goods , baking , snack , ...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]",0.647059,0.690196,0.647059,0.000000,0.164706,0.149020,0.164706,0.000000,0.164706,0.105882,0.164706,0.000000
2,209.0,"[food , fruit , baked goods , breakfast , ...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",0.647059,0.658824,0.647059,0.647059,0.164706,0.439216,0.164706,0.164706,0.164706,0.141176,0.164706,0.164706
3,95.0,"[food , bread , fast food , bun , american...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",0.647059,0.243137,0.647059,0.647059,0.164706,0.125490,0.164706,0.164706,0.164706,0.070588,0.164706,0.164706
4,101.0,"[text , font , design , yellow , graphics ...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]",1,0.792157,1.000000,1.000000,1,0.737255,1.000000,1.000000,0,0.003922,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,8.0,"[text , food , snack]","[0.6588235294117647, 0.00784313725490196, 0.71...",0,"[0.5019607843137255, 0.0, 0.5019607843137255, ...","[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.5019607843137255, 0.501...",0.501961,0.658824,0.501961,0.501961,0,0.007843,0.000000,0.501961,0.501961,0.713725,0.501961,0.501961
88512,9.0,"[text , poster , cartoon]","[0.01568627450980392, 0.5098039215686274, 0.78...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",1,0.015686,1.000000,1.000000,1,0.509804,1.000000,1.000000,1,0.780392,1.000000,1.000000
88513,11.0,"[text , yellow , packaging and labeling , f...","[0.01568627450980392, 0.2235294117647059, 0.43...",0,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[1.0, 1.0, 0.0]",1,0.015686,0.000000,1.000000,1,0.223529,0.000000,1.000000,0,0.439216,1.000000,0.000000
88514,0.0,"[sport , person , barbell , physical fitnes...","[0.6862745098039216, 0.5215686274509804, 0.109...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",1,0.686275,1.000000,1.000000,1,0.521569,1.000000,1.000000,1,0.109804,1.000000,1.000000


In [28]:
dropped = ['accent_color', 'dominant_colors', 'bg_color','fore_color']
data = df1.drop(dropped,axis=1)

In [66]:
data

,0,1,2,3,4,5,6,7,8,9,...,204,205,206,207,208,209,210,211,212,213
0,0.11669320256138842,-0.02748321322724223,-0.18671171832829714,0.13605108934765062,0.041623774236844234,-0.04426240587296585,-0.16804125905036926,0.0029527731239795685,-0.1443632487207651,-0.1326462415357431,...,1.0,1.0,0.7529411764705882,0.00784313725490196,0.7529411764705882,0.7529411764705882,0.796078431372549,0.00784313725490196,0.796078431372549,0.796078431372549
1,0.05400834072913442,0.09934965574315616,-0.2152196911296674,0.02954484788434846,0.06073764116237206,0.14323928326900517,-0.12100933319223779,-0.18622600233980588,-0.13140801593960663,0.15921796445867845,...,0.6470588235294118,0.0,0.16470588235294117,0.14901960784313725,0.16470588235294117,0.0,0.16470588235294117,0.10588235294117647,0.16470588235294117,0.0
2,0.02925444227692328,0.19349391189844986,-0.20249813701957464,-0.1028827052367361,-0.059571249116408195,0.04836817360238025,-0.08720943400342214,-0.18481898856790444,-0.13208241447022087,0.19195087803037544,...,0.6470588235294118,0.6470588235294118,0.16470588235294117,0.4392156862745098,0.16470588235294117,0.16470588235294117,0.16470588235294117,0.1411764705882353,0.16470588235294117,0.16470588235294117
3,-0.07980210549301571,0.20305788061685032,-0.16652481506268182,-0.0060051475134160785,-0.1397709906101227,0.03020856488082144,-0.00533743587632974,-0.11638561534022705,-0.14578692979800204,0.1541647789378961,...,0.6470588235294118,0.6470588235294118,0.16470588235294117,0.12549019607843137,0.16470588235294117,0.16470588235294117,0.16470588235294117,0.07058823529411765,0.16470588235294117,0.16470588235294117
4,0.09868416807148606,-0.0001471476862207055,-0.209817468887195,0.15010760573204607,0.03032800830260385,-0.059315741586033255,-0.12231099046766758,0.007357252994552255,-0.17132411850616336,-0.09631121065467596,...,1.0,1.0,1.0,0.7372549019607844,1.0,1.0,0.0,0.00392156862745098,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,0.10904438731571038,0.23766749103864035,-0.22817798455556235,0.09054082135359447,0.06305411458015442,0.04986237486203512,-0.15475784987211227,-0.12897060811519623,-0.13423940539360046,0.1439950962861379,...,0.5019607843137255,0.5019607843137255,0.0,0.00784313725490196,0.0,0.5019607843137255,0.5019607843137255,0.7137254901960784,0.5019607843137255,0.5019607843137255
88512,0.13867605229218802,-0.0037125994761784873,-0.2738087773323059,0.17919704814751944,0.06553142269452412,0.055552112559477486,-0.13970170666774115,0.012120584646860758,-0.0706338385740916,-0.17047056555747986,...,1.0,1.0,1.0,0.5098039215686274,1.0,1.0,1.0,0.7803921568627451,1.0,1.0
88513,-0.12987129162583086,0.1158151196108924,-0.3315940300623576,-0.07978270658188397,0.13186225212282604,-0.06317237267891566,-0.13333137623137897,-0.025478681549429893,-0.23176068564256033,0.033702126807636686,...,0.0,1.0,1.0,0.2235294117647059,0.0,1.0,0.0,0.4392156862745098,1.0,0.0
88514,0.11238148869438605,-0.11996569819748401,-0.1949817560951818,0.07207865224812518,0.01519832055677067,-0.0688453597439961,0.021324452046643603,-0.1681436273862015,-0.1679826631803404,-0.025150730046020312,...,1.0,1.0,1.0,0.5215686274509804,1.0,1.0,1.0,0.10980392156862745,1.0,1.0


In [29]:
d2=data

In [68]:
d2

,likes,tags,is_bw,dominant_colors_R,accent_color_R,bg_color_R,fore_color_R,dominant_colors_G,accent_color_G,bg_color_G,fore_color_G,dominant_colors_B,accent_color_B,bg_color_B,fore_color_B
0,29.0,"[text , font , graphics , screenshot , gra...",0,1,0.788235,1.000000,1.000000,0.752941,0.007843,0.752941,0.752941,0.796078,0.007843,0.796078,0.796078
1,44.0,"[dessert , baked goods , baking , snack , ...",0,0.647059,0.690196,0.647059,0.000000,0.164706,0.149020,0.164706,0.000000,0.164706,0.105882,0.164706,0.000000
2,209.0,"[food , fruit , baked goods , breakfast , ...",0,0.647059,0.658824,0.647059,0.647059,0.164706,0.439216,0.164706,0.164706,0.164706,0.141176,0.164706,0.164706
3,95.0,"[food , bread , fast food , bun , american...",0,0.647059,0.243137,0.647059,0.647059,0.164706,0.125490,0.164706,0.164706,0.164706,0.070588,0.164706,0.164706
4,101.0,"[text , font , design , yellow , graphics ...",0,1,0.792157,1.000000,1.000000,1,0.737255,1.000000,1.000000,0,0.003922,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,8.0,"[text , food , snack]",0,0.501961,0.658824,0.501961,0.501961,0,0.007843,0.000000,0.501961,0.501961,0.713725,0.501961,0.501961
88512,9.0,"[text , poster , cartoon]",0,1,0.015686,1.000000,1.000000,1,0.509804,1.000000,1.000000,1,0.780392,1.000000,1.000000
88513,11.0,"[text , yellow , packaging and labeling , f...",0,1,0.015686,0.000000,1.000000,1,0.223529,0.000000,1.000000,0,0.439216,1.000000,0.000000
88514,0.0,"[sport , person , barbell , physical fitnes...",0,1,0.686275,1.000000,1.000000,1,0.521569,1.000000,1.000000,1,0.109804,1.000000,1.000000


In [30]:
data2=data.drop('tags',1)

In [31]:
data22=data2.astype('object')

In [32]:
type(data2)

pandas.core.frame.DataFrame

In [33]:
data22.dtypes

likes                object
is_bw                object
dominant_colors_R    object
accent_color_R       object
bg_color_R           object
fore_color_R         object
dominant_colors_G    object
accent_color_G       object
bg_color_G           object
fore_color_G         object
dominant_colors_B    object
accent_color_B       object
bg_color_B           object
fore_color_B         object
dtype: object

In [71]:
d2['tags']=d2['tags'].apply(lambda txt: text_prepare(str(txt)[1:-1]))
tokenized_tag = d2['tags'].apply(lambda x: x.split()[1:-1])
tokenized_tag[1]

[',',
 'baked',
 'goods',
 ',',
 'baking',
 ',',
 'snack',
 ',',
 'food',
 ',',
 'sweetness',
 ',',
 'finger',
 'food',
 ',',
 'bakery',
 ',',
 'petit',
 'four',
 ',',
 'muffin',
 ',',
 'indoor',
 ',',
 'kue',
 ',']

##FastText

In [72]:
modelFastTextSkipGram = FastText(tokenized_tag, 
                     size=200, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2.  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=1  # 1 for Skipgram model
                     )

modelFastTextSkipGram.build_vocab(tokenized_tag, update=True)
modelFastTextSkipGram.train(tokenized_tag, total_examples= len(tokenized_tag), epochs=20)

INFO - 18:07:22: collecting all words and their counts
INFO - 18:07:22: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 18:07:22: PROGRESS: at sentence #10000, processed 184916 words, keeping 2209 word types
INFO - 18:07:22: PROGRESS: at sentence #20000, processed 375164 words, keeping 2671 word types
INFO - 18:07:22: PROGRESS: at sentence #30000, processed 562939 words, keeping 2938 word types
INFO - 18:07:22: PROGRESS: at sentence #40000, processed 754746 words, keeping 3128 word types
INFO - 18:07:22: PROGRESS: at sentence #50000, processed 941580 words, keeping 3261 word types
INFO - 18:07:22: PROGRESS: at sentence #60000, processed 1134158 words, keeping 3405 word types
INFO - 18:07:22: PROGRESS: at sentence #70000, processed 1323170 words, keeping 3506 word types
INFO - 18:07:22: PROGRESS: at sentence #80000, processed 1513335 words, keeping 3611 word types
INFO - 18:07:23: collected 3667 word types from a corpus of 1673812 raw words and 88516 sentences
I

In [73]:
modelFastTextSkipGram.save(path+ "Save Models\\200_fastText_Skipgram_.model")

INFO - 18:11:12: saving FastText object under C:\Users\jyoti\Desktop\NLP\NLP_project\NLP-project-\Save Models\200_fastText_Skipgram_.model, separately None
INFO - 18:11:12: not storing attribute vectors_norm
INFO - 18:11:12: not storing attribute vectors_vocab_norm
INFO - 18:11:12: not storing attribute vectors_ngrams_norm
INFO - 18:11:12: not storing attribute buckets_word
INFO - 18:11:13: saved C:\Users\jyoti\Desktop\NLP\NLP_project\NLP-project-\Save Models\200_fastText_Skipgram_.model


In [75]:
modelFastTextSkipGram.wv.most_similar("Gastroenteritis")

[('nunatak', 0.740767776966095),
 ('tagliatelle', 0.7130957841873169),
 ('3d', 0.6915515065193176),
 ('alps', 0.6907766461372375),
 ('crispy', 0.6732327938079834),
 ('phalaenopsis', 0.6709675788879395),
 ('mehndi', 0.6667892336845398),
 ('x', 0.6666353940963745),
 ('karaage', 0.6663018465042114),
 ('falafel', 0.6660338640213013)]

In [76]:
modelFastTextSkipGram.wv.most_similar("pizza")

[('flatbread', 0.7030203342437744),
 ('manakish', 0.6845539808273315),
 ('pepperoni', 0.6843120455741882),
 ('italian', 0.6694173812866211),
 ('casserole', 0.6623860597610474),
 ('sicilian', 0.6587257385253906),
 ('california', 0.6477446556091309),
 ('macaroni', 0.6166598200798035),
 ('cheddar', 0.6155270338058472),
 ('cassolette', 0.6130914688110352)]

In [77]:
def word_vector(tokens, size,model):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [78]:
fastText_arrays = np.zeros((len(tokenized_tag), 200)) 
for i in range(len(tokenized_tag)):
    fastText_arrays[i,:] = word_vector(tokenized_tag[i], 200,modelFastTextSkipGram)
fastText_df = pd.DataFrame(fastText_arrays)
fastText_df.shape

C:\Users\jyoti\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(88516, 200)

In [79]:
type(data2)

pandas.core.frame.DataFrame

In [80]:
dataset_fastText2 = pd.DataFrame(np.hstack((fastText_df,data22)))

In [81]:
dataset_fastText2

,0,1,2,3,4,5,6,7,8,9,...,204,205,206,207,208,209,210,211,212,213
0,0.0207659,-0.233643,-0.223695,0.175525,0.156854,-0.00873783,-0.0605019,0.0642362,-0.297105,-0.164832,...,1,1,0.752941,0.00784314,0.752941,0.752941,0.796078,0.00784314,0.796078,0.796078
1,-0.210699,-0.165044,-0.0573634,0.0164191,0.0419501,-0.0900447,-0.157834,-0.137588,-0.0736064,-0.121538,...,0.647059,0,0.164706,0.14902,0.164706,0,0.164706,0.105882,0.164706,0
2,-0.134274,-0.185613,-0.119694,0.0339838,-0.0216417,-0.00807037,-0.150548,-0.193324,-0.0955016,-0.107727,...,0.647059,0.647059,0.164706,0.439216,0.164706,0.164706,0.164706,0.141176,0.164706,0.164706
3,-0.0426015,-0.228787,-0.0988958,-0.0359875,0.00808325,0.0377704,-0.187283,-0.168977,-0.0970396,-0.0807703,...,0.647059,0.647059,0.164706,0.12549,0.164706,0.164706,0.164706,0.0705882,0.164706,0.164706
4,-0.0107071,-0.19839,-0.20092,0.143079,0.126162,0.00587593,-0.0899024,0.00097935,-0.280599,-0.152933,...,1,1,1,0.737255,1,1,0,0.00392157,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,-0.150323,-0.213728,-0.0557028,-0.137019,-0.0675649,-0.0560865,-0.0527602,-0.219713,-0.0837514,-0.136663,...,0.501961,0.501961,0,0.00784314,0,0.501961,0.501961,0.713725,0.501961,0.501961
88512,0.037114,-0.165513,-0.153659,-0.0058311,0.102352,0.128333,-0.0746,-0.0991168,-0.183549,-0.166253,...,1,1,1,0.509804,1,1,1,0.780392,1,1
88513,-0.136941,-0.121858,-0.0278116,-0.0606348,-0.137365,0.0663165,0.0572796,-0.00543965,-0.206571,-0.0980717,...,0,1,1,0.223529,0,1,0,0.439216,1,0
88514,-0.0980418,-0.062092,-0.257851,0.0285083,0.0532889,0.00466773,-0.107588,-0.00226749,-0.172921,-0.160602,...,1,1,1,0.521569,1,1,1,0.109804,1,1


## Train test split

In [82]:
data=dataset_fastText2

In [83]:
# from sklearn import preprocessing
# w2v_df = pd.DataFrame(data)
# data=w2v_df.astype(str)

# normalizer = preprocessing.MinMaxScaler(feature_range=(0, 1))
# w2v_df = pd.DataFrame(normalizer.fit_transform(data),  columns = data.columns)

In [84]:
# data2

In [85]:
# #Converting the columns into the float datatype
# a=['dominant_colors_R','dominant_colors_G','dominant_colors_B']
# for i in a:
#   data2[i]= data2[i].astype(str).astype(float)
# data

In [86]:
type(data)

pandas.core.frame.DataFrame

In [89]:
data.dtypes

0      object
1      object
2      object
3      object
4      object
        ...  
209    object
210    object
211    object
212    object
213    object
Length: 214, dtype: object

In [87]:
from sklearn import preprocessing
df = pd.DataFrame(data)
data=df.astype(str)

normalizer = preprocessing.MinMaxScaler(feature_range=(0, 1))
normalized_df = pd.DataFrame(normalizer.fit_transform(data),  columns = data.columns)

ValueError: could not convert string to float: '[]'

In [ ]:
# from sklearn import preprocessing

# bow_data1 = preprocessing.MinMaxScaler(feature_range=(0, 1))
# bow_normalized_df = pd.DataFrame(bow_data1.fit_transform(data),  columns = data.columns)

In [ ]:
# bow_normalized_df

In [ ]:
# data.astype(str)

In [ ]:
y=data[200]
X1=data.drop(200,1)

In [ ]:
type(X1)

pandas.core.frame.DataFrame

In [ ]:
type(y)

pandas.core.series.Series

In [ ]:
y.shape

(88516,)

In [ ]:
X1.shape

(88516, 213)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=2022)

###XGBoost model

In [ ]:
# #XGBoost hyper-parameter tuning
# def hyperParameterTuning(X_train, y_train):
#     param_tuning = {
#         'learning_rate': [0.01, 0.1],
#         'max_depth': [3, 5, 7, 10],
#         'min_child_weight': [1, 3, 5],
#         'subsample': [0.5, 0.7],
#         'colsample_bytree': [0.5, 0.7],
#         'n_estimators' : [100, 200, 500],
#         'objective': ['reg:squarederror']
#     }

#     xgb_model = XGBRegressor()

#     gsearch = GridSearchCV(estimator = xgb_model,
#                            param_grid = param_tuning,                        
#                            #scoring = 'neg_mean_absolute_error', #MAE
#                            #scoring = 'neg_mean_squared_error',  #MSE
#                            cv = 5,
#                            n_jobs = -1,
#                            verbose = 1)

#     gsearch.fit(X_train,y_train)

#     return gsearch.best_params_

In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
        enable_categorical=True,
        objective = 'reg:squarederror',
        colsample_bytree = 0.5,
        learning_rate = 0.05,
        max_depth = 6,
        min_child_weight = 1,
        n_estimators = 1000,
        subsample = 0.7)

xgb_model.fit(x_train, y_train)
# y_pred_xgb = xgb_model.fit(x_train, np.ravel(y_train, order='C'))


ValueError: DataFrame.dtypes for data must be int, float, bool or category.  When
categorical type is supplied, DMatrix parameter `enable_categorical` must
be set to `True`. Invalid columns:0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 

In [ ]:
y_pred_xgb = xgb_model.predict(x_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print("RMSE: %f" % (rmse))
R2=r2_score(y_test,y_pred_xgb)
print("R2 Score: %f" % (R2))
print()

In [ ]:
def plot_feature_importance(importance,names,model_type):

#Create arrays from feature importance and feature names
  feature_importance = np.array(importance)
  feature_names = np.array(names)

  #Create a DataFrame using a Dictionary
  data={'feature_names':feature_names,'feature_importance':feature_importance}
  fi_df = pd.DataFrame(data)

  #Sort the DataFrame in order decreasing feature importance
  fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

  #Define size of bar plot
  plt.figure(figsize=(20,20))
  #Plot Searborn bar chart
  sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
  #Add chart labels
  plt.title(model_type + 'FEATURE IMPORTANCE')
  plt.xlabel('FEATURE IMPORTANCE')
  plt.ylabel('FEATURE NAMES')

plot_feature_importance(xg_reg.feature_importances_,x_train.columns,'XG BOOST')